In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from urlAnalysis import create_graph, plot_URL_decay

In [ ]:
df = pd.DataFrame(urls)
df.sort_values('in_degree', ascending=False)
#df['netloc'] = df.apply(lambda r: get_url_domain(r['out_url']), axis=1)

In [ ]:
df['netloc'] = df.apply(lambda r: get_url_domain(r['out_url']), axis=1)

# Institutions

In [ ]:
print('Most popular Institutions')
inst.groupby('Institution').mean()['popularity'].sort_values(ascending=False)[:20]

In [ ]:
inst.groupby('Institution').mean().plot.scatter(x='Score', y='popularity')

In [ ]:
corr = inst.groupby('Institution').mean()[['popularity', 'World Rank', 'National Rank', 'Alumni Employment', 'Publications', 'Influence', 'Citations', 'Broad Impact', 'Patents', 'Score']].corr()
#sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values)
corr.iloc[0]

In [ ]:

countries['Name'] = countries['Name'].map(lambda n: n+'_user')
countries['Location'] = countries['Location'].map(lambda n: n+'_inst')

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite
B = nx.Graph()
B.add_edges_from([(row['Name'], row['Location']) for _, row in countries.iterrows()])
plt.figure(figsize=(10,10))
X, Y = bipartite.sets(B)
pos = dict()
pos.update( (n, (1, i)) for i, n in enumerate(X) ) # put nodes from X at x=1
pos.update( (n, (2, i*4)) for i, n in enumerate(Y) ) # put nodes from Y at x=2
nx.draw(B, pos=pos, with_labels = True)

# Repositories

In [ ]:
repos.groupby('Field').size().sort_values(ascending=False)